**Постановка задачи:**

1. Собрать данные о доходах и данные об аудитории сайта в единый отчёт.
2. Посчитать метрику ARPU по каждому устройству.
3. Найти модель устройства с наибольшим ARPU.

В качестве источников информации инспользуется данные, полученные по API-запросу и данные CSV-файла
.



---



Выполняю инсталяцию и импорт библиотек, которые могут понадобиться в ходе выполнения задач проекта.

Для получения данных Яндекс.Метрики формирую словарь с параметрами запроса.


---




In [ ]:
! pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd 
import numpy as np
import json 
import requests
from pprint import pprint

In [ ]:
token = ' ' 
headers = {'Authorization': 'OAuth ' + token}

In [ ]:
params = {'metrics': 'ym:s:users',
          'dimensions': 'ym:s:date, ym:s:deviceCategory, ym:s:mobilePhone, ym:s:mobilePhoneModel',
          'date1': '2019-05-01',
          'date2': '2019-05-31', 
          'limit': 100000,
          'ids': 30177909}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)
metrika_data = response.json()
metrika_data

200


{'query': {'ids': [30177909],
  'dimensions': ['ym:s:date',
   'ym:s:deviceCategory',
   'ym:s:mobilePhone',
   'ym:s:mobilePhoneModel'],
  'metrics': ['ym:s:users'],
  'sort': ['-ym:s:users'],
  'date1': '2019-05-01',
  'date2': '2019-05-31',
  'limit': 100000,
  'offset': 1,
  'group': 'Week',
  'auto_group_size': '1',
  'attr_name': '',
  'quantile': '50',
  'offline_window': '21',
  'attribution': 'LastSign',
  'currency': 'RUB',
  'adfox_event_id': '0'},
 'data': [{'dimensions': [{'name': '2019-05-20'},
    {'icon_id': 'desktop',
     'icon_type': 'device',
     'name': 'PC',
     'id': 'desktop'},
    {'icon_id': None, 'icon_type': None, 'name': None, 'id': None},
    {'icon_type': None, 'name': None, 'icon_id': None}],
   'metrics': [105.0]},
  {'dimensions': [{'name': '2019-05-16'},
    {'icon_id': 'desktop',
     'icon_type': 'device',
     'name': 'PC',
     'id': 'desktop'},
    {'icon_id': None, 'icon_type': None, 'name': None, 'id': None},
    {'icon_type': None, 'name': N

Данные, полученные от API необходимо преобразовать в DataFrame.

1. Непосредственно статистические данные получаю по ключу 'data'.
2. Структурирую загруженные данные, избавившись от лишней информации и вывожу полученный DataFrame.



---



In [ ]:
result = []

for data_item in metrika_data['data']:
    new_dict = {}
    for i,dimension in enumerate(data_item['dimensions']):
        new_dict[metrika_data['query']['dimensions'][i]] = dimension['name']
    for i,metric in enumerate(data_item['metrics']):
        new_dict[metrika_data['query']['metrics'][i]] = metric
    result.append(new_dict)
result
df = pd.DataFrame(result)
print(df)

      ym:s:date ym:s:deviceCategory ym:s:mobilePhone ym:s:mobilePhoneModel  \
0    2019-05-20                  PC             None                  None   
1    2019-05-16                  PC             None                  None   
2    2019-05-22                  PC             None                  None   
3    2019-05-23                  PC             None                  None   
4    2019-05-13                  PC             None                  None   
..          ...                 ...              ...                   ...   
315  2019-05-31         Smartphones          Samsung             Galaxy J5   
316  2019-05-31         Smartphones           Huawei               STF-L09   
317  2019-05-31         Smartphones           Huawei               VTR-L29   
318  2019-05-31         Smartphones           Xiaomi          Redmi 5 Plus   
319  2019-05-31         Smartphones           Xiaomi          Redmi Note 4   

     ym:s:users  
0         105.0  
1          94.0  
2        

Проверяю информацию по заполняемости данными в полученном DataFrame.


---




In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ym:s:date              320 non-null    object 
 1   ym:s:deviceCategory    320 non-null    object 
 2   ym:s:mobilePhone       280 non-null    object 
 3   ym:s:mobilePhoneModel  284 non-null    object 
 4   ym:s:users             320 non-null    float64
dtypes: float64(1), object(4)
memory usage: 12.6+ KB


Произвожу переименование колонок.


---



In [ ]:
df.rename(columns={'ym:s:date': 'date', 'ym:s:deviceCategory' : 'deviceCategory', 'ym:s:mobilePhone' : 'mobilePhone', 'ym:s:mobilePhoneModel' : 'mobilePhoneModel', 'ym:s:users' : 'users'}, inplace=True)
df

,date,deviceCategory,mobilePhone,mobilePhoneModel,users
0,2019-05-20,PC,None,None,105.0
1,2019-05-16,PC,None,None,94.0
2,2019-05-22,PC,None,None,94.0
3,2019-05-23,PC,None,None,92.0
4,2019-05-13,PC,None,None,91.0
...,...,...,...,...,...
315,2019-05-31,Smartphones,Samsung,Galaxy J5,1.0
316,2019-05-31,Smartphones,Huawei,STF-L09,1.0
317,2019-05-31,Smartphones,Huawei,VTR-L29,1.0
318,2019-05-31,Smartphones,Xiaomi,Redmi 5 Plus,1.0


Избавляюсь от None в ячейках DataFrame.


---



In [ ]:
df_no_na = df.fillna(value = 'device')
df_no_na

,date,deviceCategory,mobilePhone,mobilePhoneModel,users
0,2019-05-20,PC,device,device,105.0
1,2019-05-16,PC,device,device,94.0
2,2019-05-22,PC,device,device,94.0
3,2019-05-23,PC,device,device,92.0
4,2019-05-13,PC,device,device,91.0
...,...,...,...,...,...
315,2019-05-31,Smartphones,Samsung,Galaxy J5,1.0
316,2019-05-31,Smartphones,Huawei,STF-L09,1.0
317,2019-05-31,Smartphones,Huawei,VTR-L29,1.0
318,2019-05-31,Smartphones,Xiaomi,Redmi 5 Plus,1.0


Перед тем как совместить данные из отчёта Яндекс.Метрики и данные CSV-файла:

1. Группирую данные из отчёта Яндекс.Метрики по 'deviceCategory', 'mobilePhone', 'mobilePhoneModel' и суммирую количество пользователей для каждого типа устройств, посетивших сайт в течение месяца.

2. После группировки оставляю по условию задания только те строки получившейся таблицы, в которых оказалось трое и более пользователей.



---



In [ ]:
grouped_df_copy = df_no_na.groupby(['deviceCategory', 'mobilePhone','mobilePhoneModel'])[['users']].sum()
df1 = grouped_df_copy[(grouped_df_copy['users'] >= 3)]
df1

users
deviceCategory mobilePhone mobilePhoneModel         
PC             device      device             1966.0
Smartphones    ASUS        ZC554KL               3.0
               Apple       iPhone              118.0
               Huawei      AUM-L29               3.0
                           CLT-L29               3.0
                           COL-L29               3.0
                           LLD-L31               4.0
                           STF-L09               5.0
               Meizu       16th                  3.0
               Samsung     Galaxy A5             7.0
                           Galaxy J7             3.0
                           Galaxy S7             5.0
                           Galaxy S8             6.0
                           SAMSUNG SM-G935W8     3.0
                           SM-G965F              4.0
                           SM-N960F              4.0
               Sony        G3112                 3.0
               Xiaomi      MI 8                  3.0
                           MI 8 SE               3.0
                           Mi A1                 7.0
                           Mi A2 Lite            3.0
                           Redmi 4X              5.0
                           Redmi 5 Plus          3.0
                           Redmi 5A              3.0
                           Redmi Note 3          3.0
                           Redmi Note 4          7.0
                           Redmi Note 4X         4.0
                           Redmi Note 5          7.0
                           Redmi Note 5A         3.0
               device      device                5.0
Tablets        Apple       iPad                 13.0

Получаю информацию об обработанном DataFrame, чтобы убедиться в готовности DataFrame для объединения с CSV-файлом и извленения финальных данных по исходному заданию к проекту



---



In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 31 entries, ('PC', 'device', 'device') to ('Tablets', 'Apple', 'iPad')
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   users   31 non-null     float64
dtypes: float64(1)
memory usage: 1.9+ KB


Загружаю CSV-файл в качестве дополнительного источника информации. И объеденяю его по общим колонкам с DataFrame, полученным по API-запросу ранее. Считаю значение ARPU для каждой строки. Получаю колонку с APPU и присоеденяю её к объёденённому DataFrame. 


---



In [ ]:
df2 = pd.read_csv('revenue_data.csv')
res = df2.merge(df1, on=['deviceCategory', 'mobilePhone', 'mobilePhoneModel'])
res_ARPU = res['revenue'] / res['users']
res['ARPU'] = res_ARPU
res

,deviceCategory,mobilePhone,mobilePhoneModel,revenue,users,ARPU
0,Smartphones,ASUS,ZC554KL,1.130402,3.0,0.376801
1,Smartphones,Apple,iPhone,0.972879,118.0,0.008245
2,Smartphones,Huawei,AUM-L29,2.388569,3.0,0.796190
3,Smartphones,Huawei,CLT-L29,3.470900,3.0,1.156967
4,Smartphones,Huawei,COL-L29,2.758386,3.0,0.919462
5,Smartphones,Huawei,LLD-L31,5.220878,4.0,1.305220
6,Smartphones,Huawei,STF-L09,3.272568,5.0,0.654514
7,Smartphones,Meizu,16th,1.087090,3.0,0.362363
8,Smartphones,Samsung,Galaxy A5,5.060859,7.0,0.722980
9,Smartphones,Samsung,Galaxy J7,2.196402,3.0,0.732134


Финальный этап. Нахожу максимальное значение ARPU в соответствующей колонке и по индексу строки вывожу данные устройства с макимальным ARPU.

**Поставленная задача выполнена.**



---




In [ ]:
res['ARPU'].idxmax()

27

In [ ]:
max_ARPU = res.iloc[27]
max_ARPU

deviceCategory        Smartphones
mobilePhone                Xiaomi
mobilePhoneModel    Redmi Note 5A
revenue                  4.240824
users                         3.0
ARPU                     1.413608
Name: 27, dtype: object